In [1]:
import torch
import numpy as np
import torch.nn as nn
import os
import sys

In [2]:
notebook_path = os.getcwd()
parent_dir = os.path.dirname(notebook_path)
sys.path.append(parent_dir)

In [3]:
import importnb
with __import__('importnb').Notebook(): 
    from tools import ScaledDotProductAttention
    from tools import MultiHeadAttention
    from tools import AddPositionalEncoding
    from tools import TransformerFFN
    from notebooks.model import TransformerEncoderLayer,TransformerEncoder

## SelfAttentionの実行

### 適当なデータの作成
---
新しくデータkを作る。kは**(B,T,d_model)**のshapeを持つテンソルである。


### SelfAttentionにする
---
q,k,vを同じテンソルにすることでSelfAttentionにする。


### forwardで計算を行う。
---
呼び出したspa.forward()によって計算を行う。
この時spaの初期化で与えるd_kには本来d_modelをhead数で割った値が入る(デフォルトだと512/8で64)

In [60]:
###SelfAttentionの実行
num_head = 4
batch_size = 2
seq_len = 8
d_model = 64
max_len = 512
d_ff =1024
N = 6
pad_idx = 0
vocab_size = 4200
dropout_rate=0.1
layer_norm_eps = 1e-5
k = torch.randint(0,2000, (batch_size, seq_len)).to(torch.int64)

## MultiHeadAttentionの実行
ランダムなテンソル、スタンダードなマスクを使う。

In [61]:
def create_incremental_mask(seq_len):
    """
    seq_len x seq_len のサイズのマスクを生成する。
    0列目は全てFalse、以降の列では上から順にTrueの数を増やしていく。
    """
    # seq_len x seq_len の行列を生成し、初期値は全てFalseに設定
    mask = torch.full((seq_len, seq_len), False)

    # 各列に対して、上から順にTrueをセットする
    for i in range(seq_len):
        mask[:i, i] = True

    return mask
mask = create_incremental_mask(seq_len).repeat(batch_size,1,1)
mask.shape

torch.Size([2, 8, 8])

In [62]:
encoder = TransformerEncoder(
    d_model = d_model,
    d_ff = d_ff,
    num_head = num_head,
    N=N,
    max_len=max_len,
    pad_idx=pad_idx,
    vocab_size=vocab_size,
    dropout_rate=dropout_rate,
    layer_norm_eps=layer_norm_eps
)

In [63]:
output = encoder(
    k.to(torch.int32),
    mask=mask)

torch.Size([2, 8, 64])


In [64]:
output.shape

torch.Size([2, 8, 64])

In [65]:
output

tensor([[[-0.4391, -1.8515, -0.8418,  ...,  0.6907, -0.0847,  0.8816],
         [ 3.0435,  0.5011, -0.1033,  ...,  1.4353,  0.1848, -0.4028],
         [ 0.8422, -1.1325, -0.8021,  ..., -1.2472, -1.3949,  0.6650],
         ...,
         [ 2.2137, -2.7385,  0.3143,  ...,  0.3130, -1.5519, -0.7651],
         [ 1.7087, -1.1008,  0.9134,  ...,  0.3804,  0.2236, -0.2529],
         [ 1.3546,  0.4397, -0.5981,  ...,  0.5595,  0.7469,  0.1898]],

        [[ 2.6964, -1.3504, -0.6953,  ...,  1.2061, -0.8188,  0.4482],
         [-0.9991, -1.7268,  1.4084,  ...,  1.4026, -0.7045, -0.0687],
         [-0.0136, -1.0802,  2.0668,  ...,  1.7309, -0.5650, -0.9085],
         ...,
         [ 1.2171, -0.9127, -0.3268,  ...,  1.0246,  1.2878, -0.7483],
         [ 0.2356, -0.0503,  1.0148,  ...,  0.1918, -0.0551, -0.0698],
         [ 0.1326, -0.8866,  0.5033,  ..., -0.7366, -1.3109,  1.6053]]],
       grad_fn=<NativeLayerNormBackward0>)

In [66]:
output.shape

torch.Size([2, 8, 64])